In [ ]:
import torch
import pandas as pd
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import SGD

In [ ]:
df = pd.read_excel('./数据集.xlsx')

In [ ]:
train_x = df.iloc[:, 0].to_numpy().reshape(319, 1)
train_y = df.iloc[:, 1].to_numpy().reshape(319, 1)
target = df.iloc[:, 2].to_numpy()
x_lim = train_x.max()
y_lim = train_y.max()

In [ ]:
N = 4
class Model(nn.Module):
    def __init__(self, N):
        super(Model, self).__init__()
        self.N = N
        self.x0 = Variable(torch.rand(N), requires_grad=True)
        self.y0 = Variable(torch.rand(N), requires_grad=True)
        self.k = Variable(torch.rand(N), requires_grad=True)
        self.sigma = Variable(torch.rand(N), requires_grad=True)
        self.__parameters = dict(x0=self.x0, y0=self.y0, k=self.k, sigma=self.sigma)
        
    def forward(self, x, y, z):
        loss = 0.0
        for j in range(self.N):
            miu = ((x - self.x0[j]) * (x - self.x0[j]) + (y - self.y0[j]) * (y - self.y0[j])) / self.sigma[j]
            output = self.k[j] * torch.exp(-miu/2)
            loss += 0.5 * (output - z) * (output - z)
        return loss

    def parameters(self):
        for name, param in self.__parameters.items():
            yield param

In [ ]:
model = Model(N)
epochs = 50
lr = 0.05
optimizer = SGD(model.parameters(), lr=lr)

In [ ]:
for epoch in range(epochs):
    print(f"====={epoch}=====")
    loss = 0.0
    for i in range(len(train_x)):
        x = torch.tensor(train_x[i])
        y = torch.tensor(train_y[i])
        z = torch.tensor(target[i])
        # 归一化
        x = x / x_lim
        y = y / y_lim
        loss += model(x, y, z)
    
    optimizer.zero_grad()
    loss = loss / len(train_x)
    loss.backward()
    optimizer.step()

    # # 更新参数(手写梯度下降)
    # model.x0.data = model.x0.data - lr * model.x0.grad.data
    # model.y0.data = model.y0.data - lr * model.y0.grad.data

    # model.x0.data.clamp_(0, 28655)
    # model.y0.data.clamp_(0, 18450)

    # model.k.data = model.k.data - lr * model.k.grad.data
    # model.sigma.data = model.sigma.data - lr * model.sigma.grad.data

    # model.x0.grad.zero_()
    # model.y0.grad.zero_()
    # model.k.grad.zero_()
    # model.sigma.grad.zero_()

    # 打印loss
    print(loss.item())

In [ ]:
print(model.x0*x_lim)
print(model.y0*y_lim)